In [1]:
#default_exp pdUtils

# pd Utils
utilities function for pandas dataframe and series

In [8]:
#export
import pandas as pd
from hashlib import sha1
from nicHelper.dictUtil import saveStringToFile, loadStringFromFile
from pynamodb.constants import BINARY
from pynamodb.attributes import Attribute, UnicodeAttribute
from pynamodb.models import Model
from s3bz.s3bz import S3
import os

In [3]:
df = pd.DataFrame({'hello':[1,2,3,4,5,5]})
df

,hello
0,1
1,2
2,3
3,4
4,5
5,5


In [4]:
#export
def getDfHash(df:pd.DataFrame):
  df.to_feather('/tmp/feather')
  with open('/tmp/feather', 'rb') as f:
    objHash = sha1(f.read()).hexdigest()
  return objHash

In [5]:
%%time
getDfHash(df)

CPU times: user 13.6 ms, sys: 7.6 ms, total: 21.2 ms
Wall time: 20.3 ms


'068df0811eb710aa82148159e389157000f0b023'

## Local cache and hash

In [6]:
#export
def saveLocalCache( data:pd.DataFrame, path = '/tmp/cache'):
  saveLocalHash(data, path=path)
  return data.to_feather(path)
def saveLocalHash( data:pd.DataFrame, path = '/tmp/hash'):
  dataHash = getDfHash(data)
  saveStringToFile(dataHash,path)
def loadLocalCache( path = '/tmp/cache'):
  if not os.path.exists(path): raise Exception('cache doesnt exist')
  return pd.read_feather(path)
def loadLocalHash( path = '/tmp/hash'):
  if not os.path.exists(path): raise Exception('hash doesnt exist')
  return loadStringFromFile(path)

In [7]:
%time saveLocalCache(df)
%time saveLocalHash(df)
%time print(loadLocalHash())
%time loadLocalCache()

CPU times: user 3.16 ms, sys: 0 ns, total: 3.16 ms
Wall time: 4.09 ms
CPU times: user 1.88 ms, sys: 0 ns, total: 1.88 ms
Wall time: 1.47 ms
068df0811eb710aa82148159e389157000f0b023
CPU times: user 701 µs, sys: 0 ns, total: 701 µs
Wall time: 510 µs
CPU times: user 0 ns, sys: 2.29 ms, total: 2.29 ms
Wall time: 2.25 ms


,hello
0,1
1,2
2,3
3,4
4,5
5,5


## Remote cache and hash

In [16]:
#export
def saveRemoteHash(data:pd.DataFrame, key='', bucket='', **kwargs):
  hashKey = f'{key}-hash'
  hashString = getDfHash(data)
  dictToSave= {'hash': hashString }
  print(f'hashKey is {hashKey}')
  print('saving hash to s3')
  S3.save(key=hashKey,objectToSave=dictToSave, bucket=bucket, **kwargs )
  print(f'saved hash {hashString}')
  

def saveRemoteCache(data:pd.DataFrame, key = '', 
                    bucket = '', localCachePath='/tmp/cache', localHashPath='/tmp/hash', **kwargs):
  
  saveLocalCache(data=data, path = localCachePath)
  saveLocalHash(data=data, path = localHashPath)
  saveRemoteHash(data=data, key = key, bucket=bucket)
  S3.saveFile(key=key, path=localCachePath, bucket=bucket, **kwargs)
  
def loadRemoteCache(key='', bucket='', **kwargs):
  path = '/tmp/tmpPath'
  S3.loadFile(key,path=path ,bucket=bucket, **kwargs)
  df = pd.read_feather(path)
  return df

def loadRemoteHash(key='', bucket='', **kwargs):
  hashKey = f'{key}-hash'
  print(f'loading hashkey {hashKey}')
  loadedHash= S3.load(hashKey,bucket=bucket, **kwargs).get('hash')
  print(f'loaded hash is {loadedHash}')
  return loadedHash

In [21]:
#hide
testKey = 'testKey'
testBucket = 'villa-clipboard'
%time saveRemoteCache(df, key = testKey, bucket = testBucket)
print(loadRemoteHash(testKey, testBucket))
%timeit loadRemoteCache(key = testKey, bucket = testBucket)

hashKey is testKey-hash
saving hash to s3
saved hash 068df0811eb710aa82148159e389157000f0b023
CPU times: user 38.4 ms, sys: 7.72 ms, total: 46.2 ms
Wall time: 164 ms
loading hashkey testKey-hash
loaded hash is 068df0811eb710aa82148159e389157000f0b023
068df0811eb710aa82148159e389157000f0b023
128 ms ± 4.58 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## PynamoAttributes

In [2]:
#export
class PandasDataFrameAttribute(Attribute):
  attr_type = BINARY
  def serialize(self, value: pd.DataFrame)->bin:
    bio = BytesIO()
    value.to_feather(bio)
    data:bin = bio.getvalue()
    return data
  def deserialize(self, value: bin)->pd.DataFrame:
    bio = BytesIo(bin)
    df: pd.DataFrame = pd.read_feather(bio)
    return df

In [10]:
class Database(Model):
  class Meta:
    table_name = ''
    region = ''
    billing_mode='PAY_PER_REQUEST'
    
  brcode = UnicodeAttribute(hash_key=True, default = '')
  data = PandasDataFrameAttribute()
  
import sys
df = pd.DataFrame({'cprcode':['1234', '12345'], 'quantity':[123, 345]})
db = Database(brcode='1234', data = df)
db.data

,cprcode,quantity
0,1234,123
1,12345,345
